# II Character RNN (LSTM) 

Palyginti trumpas ir paprastas simbolio lygio RNN modelis skirtas generuoti naujus tekstą pagal įvesties tekstą.
Remiamasi šia implementacija: https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb

[re - regular expression operations](https://docs.python.org/3/library/re.html)<br>
[string - common string operations](https://docs.python.org/3/library/string.html)<br>
[unicode](https://docs.python.org/3/c-api/unicode.html)

In [ ]:
!pip install unidecode

In [ ]:
import re
import time
import torch
import random
import string
import shutils
import unidecode
import matplotlib.pyplot as plt


torch.backends.cudnn.deterministic = True

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
sys.path.append('/content/drive/My Drive/Colab Notebooks')

Keletas hyperametrų tokie kaip `TEXT_PORTION_SIZE` kuris reiškia kokio ilgio poricja yra. Čia nenaudosime `EPOCHS`, o `NUM_ITER` ir visi kiti jau naudoti tipiniai parametrai.

In [ ]:
DEVICE            = torch.device('cpu')
NUM_ITER          = 5000
HIDDEN_DIM        = 100
RANDOM_SEED       = 123
LEARNING_RATE     = 0.005
EMBEDDING_DIM     = 100
NUM_HIDDEN_LAYERS = 1
TEXT_PORTION_SIZE = 200

torch.manual_seed(RANDOM_SEED)

Konvertuojame visus ASCII simbolius, kuriuos pateikia `string.printable` ir naudosime kaip simbolių rinkinį (100 simbolių).

In [3]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [ ]:
shutil.copy('/content/drive/My Drive/Colab Notebooks/datasets/covid19_faq.txt', 'covid19_faq.txt')

Naudosime duomenų rinkinį kuris covid dažniausiai užduodami klausimai.

In [ ]:
with open('covid19_faq.txt', 'r') as f:
    textfile = f.read()

# konvertuoti specialiuosius simbolius
textfile = unidecode.unidecode(textfile)

# pašalinti papildomus tarpelius
textfile = re.sub(' +',' ', textfile)

TEXT_LENGTH = len(textfile)

print(f'Number of characters in text: {TEXT_LENGTH}')

Padaliname tekstą į mažesnes atsitiktines porcijas. Šie sakiniai gali būtų sintaksiškai ir gramatiškai neteisingai sukarpyti kas nėra gerai. Raalaus atvejo programoje norite turėti funkciją, kuri sukuria pilnus sakinius.

In [ ]:
random.seed(RANDOM_SEED)

def random_portion(textfile):
    start_index = random.randint(0, TEXT_LENGTH - TEXT_PORTION_SIZE)
    end_index = start_index + TEXT_PORTION_SIZE + 1
    return textfile[start_index:end_index]

print(random_portion(textfile))

Konvertuojame simbolius į tensorius (simbolius į skaičius), ši funkcija randa indeksus, tai simbolio `a` indeksas būtų 10.

In [ ]:
def char_to_tensor(text):
    lst = [string.printable.index(c) for c in text]
    tensor = torch.tensor(lst).long()
    return tensor

print(char_to_tensor('abcDEF'))

Atsitiktinės imties traukimas modelio mokymui. Ir taip pat ši funkcija gaus mūsų etiketes. Etiketės yra įvestis, perkeltos per vieną simbolį nes šio NN uždavinys yra nuspėti sekantį simbolį.

In [ ]:
def draw_random_sample(textfile):    
    text_long = char_to_tensor(random_portion(textfile))
    inputs = text_long[:-1]
    targets = text_long[1:]
    return inputs, targets

In [ ]:
draw_random_sample(textfile)

## Modelis 

Mūsų RNN implementacija. Naudojame `LSTMCell` klasę kuri yra labiau intuityvi nei `LSTM`.

Mūsų teksto simbolių įvesties `one hot` kodavimas šioje implemetacijoje yra skaitines reikšmes, skaičius 2, skaičius 0 ir t.t

In [ ]:
class RNN(torch.nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, output_size):
        super().__init__()
        
        # paslėptas matmuo yra 128
        self.hidden_size = hidden_size
       
        # Tai yra mūsų įterpimo sluoksnis, kuris pereina nuo sveikojo simbolio skaičiaus iki 100 dydžio įterpimo vektoriaus
        self.embed = torch.nn.Embedding(num_embeddings=input_size, embedding_dim=embed_size)
        # lstm celės kompozicija susideda iš 100 narių dydžio vektoriaus (embed_size, o paslėptas dydis yra 128 (hidden_size)
        # įterpimo sluoksnis išves 100 matmenų vektorių, o paslėptas sluoksnis bus 128 matmenų vektorius
        self.rnn = torch.nn.LSTMCell(input_size=embed_size, hidden_size=hidden_size)
        
        self.fc = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, character, hidden, cell_state):
        # tikisi simbolio dydžio [batch_size, 1]
        
        # simbolis yra duodamas įterpimo sluoksniui, batch (pakuotės) dydis yra 1
        # [batch size, embedding dim] = [1, embedding dim]
        embedded = self.embed(character)
        
        # LSTMcell duodame įterpimo vektorių (1x100) kartu su paslėta ir celės būsena iš prieš tai
        # buvusios iteracijos
        # https://i.stack.imgur.com/SjnTl.png
        (hidden, cell_state) = self.rnn(embedded, (hidden, cell_state))
        # 1. output dim: [batch size, output_size] = [1, output_size]
        # 2. hidden dim: [batch size, hidden dim]  = [1, hidden dim]
        # 3. cell dim:   [batch size, hidden dim]  = [1, hidden dim]
        
        output = self.fc(hidden)

        return output, hidden, cell_state
    
    # zero state inicializavimas
    def init_zero_state(self):
        init_hidden = torch.zeros(1, self.hidden_size).to(DEVICE)
        init_cell   = torch.zeros(1, self.hidden_size).to(DEVICE)
        return (init_hidden, init_cell)

In [ ]:
torch.manual_seed(RANDOM_SEED)

In [ ]:
model = RNN(len(string.printable), EMBEDDING_DIM, HIDDEN_DIM, len(string.printable))
model = model.to(DEVICE)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def evaluate(model, prime_str='A', predict_len=100, temperature=0.8):
    
    (hidden, cell_state) = model.init_zero_state()
    prime_input = char_to_tensor(prime_str)
    predicted = prime_str

    # sukuriame paslėptą pradinę būseną, naudojame pradinę eilutę, modelis stabilizuojasi
    for p in range(len(prime_str) - 1):
        inp = prime_input[p].unsqueeze(0)
        _, hidden, cell_state = model(inp.to(DEVICE), hidden, cell_state)
    inp = prime_input[-1].unsqueeze(0)
    
    for p in range(predict_len):

        outputs, hidden, cell_state = model(inp.to(DEVICE), hidden, cell_state)
        
        # Atsitiktinis mėginys iš tinklo kaip daugianario skirstinys (distribution)
        # inspired by https://en.wikipedia.org/wiki/Boltzmann_distribution
        output_dist = outputs.data.view(-1).div(temperature).exp() # e^{logits / T ∈[0, 1]}
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Pridėkite numatytą simbolį prie eilutės ir naudokite kaip kitą įvestį
        predicted_char = string.printable[top_i]
        predicted += predicted_char
        inp = char_to_tensor(predicted_char)

    return predicted

## Treniravimas

iteruosime 5000 kartų, tai reiškia bus 5000 žignsių

In [ ]:
start_time = time.time()

loss_list = []

for iteration in range(NUM_ITER):
    # inizializuojame zero state
    # kiekviena iteracija praeis per vieną 200 dydžio teksto dalį.
    hidden, cell_state = model.init_zero_state()
    optimizer.zero_grad()
    
    loss = 0.
    inputs, targets = draw_random_sample(textfile)
    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
    
    for c in range(TEXT_PORTION_SIZE):
        # metodas unsqueeze(0) naudojamas dėl  [batch_size, 1], pridedam tusčia dimensija
        # priminės hidden, cell_state reikšmės yra iš zero state inicializacijos bet su sekančia iteracija
        # modelis generuos naujas ir jas pernaudos rekursyviai.
        outputs, hidden, cell_state = model(inputs[c].unsqueeze(0), hidden, cell_state)
        loss += torch.nn.functional.cross_entropy(outputs, targets[c].view(1))
 
    # kainos vidurkis
    loss /= TEXT_PORTION_SIZE
    loss.backward()
    
    # atnaujiname modelio parametrus
    optimizer.step()

    #logeris
    with torch.no_grad():
        if iteration % 200 == 0:
            
            print(f'Praėjęs laika: {(time.time() - start_time)/60:.2f} min')
            print(f'Iteracija {iteration} | kaina {loss.item():.2f}\n\n')
            # vieto to kad vien tik išspaudinti kaina tai pat įvertiname modelį.
            # leidžiant jam generuoti tekstą
            print(evaluate(model, 'Th', 200), '\n')
            print(50*'=')
            
            loss_list.append(loss.item())
            plt.clf()
            plt.plot(range(len(loss_list)), loss_list)
            plt.ylabel('kaina')
            plt.xlabel('iteracija x 1000')
            plt.savefig('kaina.pdf')
            
plt.clf()
plt.ylabel('kaina')
plt.xlabel('iteracija x 1000')
plt.plot(range(len(loss_list)), loss_list)
plt.show()